In [5]:
import sys
sys.path.append("../")
import os
import json
import openai
import dotenv
import pandas as pd
from src.extract_assessment import llm_parser

dotenv.load_dotenv("../src/.env")



/home/liyu/workshop/LLM/seleya/esg_assessment/notebooks/../src/extract_assessment.py:195: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if file_name_last_year is "nan":


True

In [6]:
endpoint = os.environ.get("OPENAI_API_ENDPOINT")
api_key = os.environ.get("OPENAI_API_KEY")
api_version = os.environ.get("OPENAI_API_VERSION")
api_type = os.environ.get("OPENAI_API_TYPE")

In [20]:
def llm_parser(chat_type:str, 
               documents:str, 
               time:str=None, 
               company_name:str=None, 
               document_title:str=None,
               llm_model:str="gpt-4o"):

    if chat_type == "title_extraction":
        instruction = "The following is a document written by a company. The document is written in markdown format. Please provide the title, corporate entity and fiscal year of the document.Please output the above answer with the JSON format:{\"title\":\"xxxx\", \"corporate_entity\": \"xxxx\", \"fiscal_year\": \"xxxx\"} \
                        \nPlease output the above answer results in JSON string and no need to use markdown syntax to encapsulate JSON string into Code Block."
    
        messages = [
            {
                "role": "system",
                "content": "You are an artificial intelligence assistant, and you are better at Chinese and English conversations. You will provide users with safe, helpful, and accurate answers. At the same time, you will reject all answers related to terrorism, racial discrimination, yellow violence, and other issues.",
            },
            {
                "role": "system", 
                "content": documents},
            {
                "role": "user", 
                "content": instruction},
        ]

    elif chat_type == "assessment":
        instruction = "The ESG report title for %s is \"%s\". Please refer to the above information to read the current document and answer the following questions: \
                        \n\t1. What is the title of the current document? \
                        \n\t2. Verify if the document's company entity is the same company as %s\
                        \n\t3. Ignore the time in the title and give a similarity score between the given title and the current document title. The total score is 10 points, with 0 being the lowest and 10 being the highest. \
                        \n\t4. Ignore the time factor and rate the overlap between the current document and the provided title? The total score is 10 points, with 0 being the lowest and 10 being the highest. \
                        \n\t5. Based on the similarity between the current document title and the ESG report title for %s as well as the content of the current document, \
                            infer that  whether the ESG report framework used in the current document is consistent with the ESG report framework for %s? \
                            If so, please output the corporate entity and fiscal year of the document. If not, it is not necessary to output. \
                    \nPlease output the above answer results in JSON string and no need to use markdown syntax to encapsulate JSON string into Code Block.\
                    \nYou can refer to the following output examples: \
                            \n\t1）{\"title\": \"xxx\", \"company_match\": yes,\"title_similarity\": 10, \"similarity\": 10, \"is_ESG\": \"yes\", \"corporate_entity\": \"xxxx\", \"fiscal_year\": \"xxxx\"} \
                            \n\t2）{\"title\": \"xxx\", \"company_match\": no,\"title_similarity\": 3, \"similarity\": 2, \"is_ESG\": \"no\"}"% (company_name, document_title, company_name, company_name, company_name)
 
        messages = [
            {
                "role": "system",
                "content": "You are an helpful assistant.",
            },
            {
                "role": "system", 
                "content": documents},
            {
                "role": "user", 
                "content": instruction},
        ]

    else:
        raise ValueError("Invalid chat type")

    client = openai.AzureOpenAI(azure_endpoint=endpoint,
                                api_key=api_key,
                                api_version=api_version)
    
    try:
        chat_completion = client.chat.completions.create(
            model=llm_model,
            messages=messages
        )

        result = chat_completion.choices[0].message.content

    except Exception as e:
        raise e

    return result

In [21]:
with open(file="../resources/eric_md/000080.KS_FY2023_Sustainability Report 2023.md", mode="r") as f:
    documents = f.read()
    

In [22]:
documents[:100]

'# HITEJINRO SUSTAINABILITY REPORT 2023\n\nWE BREW HEALTHY TOMORROW\n---\nWe Brew Healthy Tomorrow\n\nHITEJ'

In [23]:
res1 = llm_parser("title_extraction", documents, llm_model="gpt-4o")

In [24]:
res1

'{"title": "HITEJINRO Sustainability Report 2023", "corporate_entity": "HITEJINRO Co., Ltd.", "fiscal_year": "2022"}'

In [25]:
json.loads(res1)

{'title': 'HITEJINRO Sustainability Report 2023',
 'corporate_entity': 'HITEJINRO Co., Ltd.',
 'fiscal_year': '2022'}

In [ ]:
input_pdf_path = "../resources/eric_md"
download_info = pd.read_excel("../resources/new_reports_240607.xlsx")

file_list = os.listdir(input_pdf_path)
index = 0

download_info["title"] = None
download_info["corporate_entity"] = None
download_info["fiscal_year"] = None

for i in range(len(download_info)):
    blob_name = download_info["path"][i]
    file_name = blob_name.split("/")[-1]
    file_name = file_name.replace(".pdf", ".md")
    if file_name in file_list:
        with open(file=os.path.join(input_pdf_path, file_name), mode="r") as f:
            documents = f.read()
        try:
            res1 = llm_parser("title_extraction", documents, llm_model="gpt-4o")
            res_json = json.loads(res1)
            download_info.loc[i, "title"] = res_json["title"]
            download_info.loc[i, "corporate_entity"] = res_json["corporate_entity"]
            download_info.loc[i, "fiscal_year"] = res_json["fiscal_year"]
            download_info.loc[i, "parser_info"] = "success"
        except Exception as e:
            download_info.loc[i, "parser_info"] = e

            
download_info.to_csv("../resources/new_reports_240607.csv")
